In [1]:
# !pip install ezc3d

In [2]:
import ezc3d
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd

In [3]:

# Load a C3D file
c3d = ezc3d.c3d('CVIP\\Project\\data\\C3D\\Data_Run_Walk\\Data_Run_Walk\\AJ026\\Session1\\Treadmill_Run\\Treadmill_Run_Comfortable\\Raw\\Treadmill_Run_Comfortable_raw.c3d')

# Basic information about the file
print(f"Number of 3D points: {c3d['data']['points'].shape[1]}")
print(f"Number of frames: {c3d['data']['points'].shape[2]}")
print(f"Point rate: {c3d['parameters']['POINT']['RATE']['value'][0]} Hz")

# Get the 3D point data
# Shape is (4, n_points, n_frames) 
# The 4 dimensions are: [x, y, z, residual] for each point
points = c3d['data']['points']
print(f"Point data shape: {points.shape}")

# Extract just x,y,z coordinates (without residuals)
xyz_data = points[:3, :, :]
print(f"XYZ data shape: {xyz_data.shape}")

Number of 3D points: 77
Number of frames: 13791
Point rate: 100.0 Hz
Point data shape: (4, 77, 13791)
XYZ data shape: (3, 77, 13791)


In [4]:
marker_data = c3d['data']['points']

In [5]:
print(marker_data.shape)

(4, 77, 13791)


In [6]:
def analyze_calibration_file(calib_path):
    """Examine the structure and content of the calibration file"""
    
    # Load calibration data (adjust based on file format)
    calib_data = pd.read_csv(calib_path, delim_whitespace=True , skiprows=5)
    
    print("=== CALIBRATION FILE ANALYSIS ===")
    print(f"Columns: {list(calib_data.columns)}")
    print(f"Shape: {calib_data.shape}")
    print(f"Time range: {calib_data['Time'].min()} to {calib_data['Time'].max()} s")
    
    # Check if it's a static trial (should have minimal movement)
    if len(calib_data) > 1:
        movement_analysis = analyze_static_trial(calib_data)
        print(f"Static trial stability: {movement_analysis}")
    
    return calib_data


In [7]:
def analyze_static_trial(data):
    """Check how stable the static trial is"""
    marker_movement = {}
    for i in range(77):  # Assuming 77 markers
        x_col = f'C_{i}_X'
        if x_col in data.columns:
            # Calculate standard deviation of marker position
            std_x = data[x_col].std()
            std_y = data[f'C_{i}_Y'].std()
            std_z = data[f'C_{i}_Z'].std()
            total_std = np.sqrt(std_x**2 + std_y**2 + std_z**2)
            if total_std > 5.0:  # Threshold in mm
                marker_movement[f'C_{i}'] = total_std
    
    return f"Unstable markers: {len(marker_movement)}"

# Analyze your calibration file


In [8]:
calib_data = analyze_calibration_file('CVIP\\Project\\data\\C3D\\Data_Run_Walk\\Data_Run_Walk\\AJ026\\Session1\\Calibration\\Calibration.csv')


C:\Users\Admin\AppData\Local\Temp\ipykernel_14700\2238574563.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  calib_data = pd.read_csv(calib_path, delim_whitespace=True , skiprows=5)


=== CALIBRATION FILE ANALYSIS ===
Columns: ['Time,LFHD,,,RFHD,,,LBHD,,,RBHD,,,LMAS,,,RMAS,,,GLAB,,,C7,,,T2,,,T10,,,CLAV,,,STRN,,,RBAK,,,LSHO,,,LUPA,,,LELB,,,LFRM,,,LWRA,,,LWRB,,,LFIN,,,RSHO,,,RUPA,,,RELB,,,RFRM,,,RWRA,,,RWRB,,,RFIN,,,LASI,,,RASI,,,LPSI,,,RPSI,,,LTHI,,,LTHAP,,,LTHAD,,,LKNE,,,LKNM,,,LTIB,,,LTIAP,,,LTIAD,,,LANK,,,LMED,,,LHEE,,,LTOE,,,LFMH,,,LSMH,,,LVMH,,,LFootOff,,,RTHI,,,RTHAP,,,RTHAD,,,RKNE,,,RKNM,,,RTIB,,,RTIAP,,,RTIAD,,,RANK,,,RMED,,,RHEE,,,RTOE,,,RFMH,,,RSMH,,,RVMH,,,RFootOff,,,TrunkIMUO,,,TrunkIMUP,,,TrunkIMUL,,,PelvisIMUO,,,PelvisIMUP,,,PelvisIMUL,,,LFemurIMUO,,,LFemurIMUP,,,LFemurIMUL,,,LTibiaIMUO,,,LTibiaIMUP,,,LTibiaIMUL,,,RFemurIMUO,,,RFemurIMUP,,,RFemurIMUL,,,RTibiaIMUO,,,RTibiaIMUP,,,RTibiaIMUL,,,PELVISO,,,PELVISA,,,PELVISL,,,PELVISP,,,LFEMURO,,,LFEMURA,,,LFEMURL,,,LFEMURP,,,LTIBIAO,,,LTIBIAA,,,LTIBIAL,,,LTIBIAP,,,LFOOTO,,,LFOOTA,,,LFOOTL,,,LFOOTP,,,LTOESO,,,LTOESA,,,LTOESL,,,LTOESP,,,RFEMURO,,,RFEMURA,,,RFEMURL,,,RFEMURP,,,RTIBIAO,,,RTIBIAA,,,RTIBIAL,,,RTIBI

KeyError: 'Time'

Debug calibration file

In [9]:
def debug_calibration_file(file_path):
    """First, let's see what's actually in your file"""
    
    print("=== DEBUGGING FILE STRUCTURE ===")
    
    # Read first 10 lines to understand the structure
    with open(file_path, 'r') as f:
        for i in range(10):
            line = f.readline().strip()
            print(f"Line {i}: {line}")
    
    # Try different reading strategies
    print("\n=== TRYING DIFFERENT READING METHODS ===")
    
    # Method 1: Read with header at row 5 (if that's where your data starts)
    try:
        data = pd.read_csv(file_path, skiprows=5)
        print(f"Method 1 - Shape: {data.shape}")
        print(f"Columns: {list(data.columns)}")
        print("First few rows:")
        print(data.head(4))
    except Exception as e:
        print(f"Method 1 failed: {e}")
    
    # Method 2: Read without headers first
    try:
        data_raw = pd.read_csv(file_path, header=None)
        print(f"\nMethod 2 - Raw shape: {data_raw.shape}")
        print("First 10 rows of raw data:")
        print(data_raw.head(10))
    except Exception as e:
        print(f"Method 2 failed: {e}")

# Debug your specific file
debug_calibration_file('CVIP\\Project\\data\\C3D\\Data_Run_Walk\\Data_Run_Walk\\AJ026\\Session1\\Calibration\\Calibration.csv')


=== DEBUGGING FILE STRUCTURE ===
Line 0: FrameNumber,40.1
Line 1: FirstFrame,1
Line 2: PointFrequency,100
Line 3: AnalogFrequency,100
Line 4: 
Line 5: Time,LFHD,,,RFHD,,,LBHD,,,RBHD,,,LMAS,,,RMAS,,,GLAB,,,C7,,,T2,,,T10,,,CLAV,,,STRN,,,RBAK,,,LSHO,,,LUPA,,,LELB,,,LFRM,,,LWRA,,,LWRB,,,LFIN,,,RSHO,,,RUPA,,,RELB,,,RFRM,,,RWRA,,,RWRB,,,RFIN,,,LASI,,,RASI,,,LPSI,,,RPSI,,,LTHI,,,LTHAP,,,LTHAD,,,LKNE,,,LKNM,,,LTIB,,,LTIAP,,,LTIAD,,,LANK,,,LMED,,,LHEE,,,LTOE,,,LFMH,,,LSMH,,,LVMH,,,LFootOff,,,RTHI,,,RTHAP,,,RTHAD,,,RKNE,,,RKNM,,,RTIB,,,RTIAP,,,RTIAD,,,RANK,,,RMED,,,RHEE,,,RTOE,,,RFMH,,,RSMH,,,RVMH,,,RFootOff,,,TrunkIMUO,,,TrunkIMUP,,,TrunkIMUL,,,PelvisIMUO,,,PelvisIMUP,,,PelvisIMUL,,,LFemurIMUO,,,LFemurIMUP,,,LFemurIMUL,,,LTibiaIMUO,,,LTibiaIMUP,,,LTibiaIMUL,,,RFemurIMUO,,,RFemurIMUP,,,RFemurIMUL,,,RTibiaIMUO,,,RTibiaIMUP,,,RTibiaIMUL,,,PELVISO,,,PELVISA,,,PELVISL,,,PELVISP,,,LFEMURO,,,LFEMURA,,,LFEMURL,,,LFEMURP,,,LTIBIAO,,,LTIBIAA,,,LTIBIAL,,,LTIBIAP,,,LFOOTO,,,LFOOTA,,,LFOOTL,,,LFOOTP,,,LTOES

Parse Vicon-style calibration file with comprehensive metadata

In [10]:
def parse_vicon_calibration_file(file_path):
    """
    Parse Vicon-style calibration file with comprehensive metadata
    """
    print("=== PARSING VICON CALIBRATION FILE ===")
    
    # Read metadata from first 5 lines
    with open(file_path, 'r') as f:
        metadata_lines = [f.readline().strip() for _ in range(5)]
    
    # Parse metadata
    metadata = {}
    for line in metadata_lines:
        if line and ',' in line:
            key, value = line.split(',', 1)
            metadata[key] = value
    
    print("Metadata:")
    for key, value in metadata.items():
        print(f"  {key}: {value}")
    
    # Read the actual data starting from row 5
    data = pd.read_csv(file_path, skiprows=5)
    
    print(f"\nData shape: {data.shape}")
    print(f"First few columns: {list(data.columns[:20])}")
    
    return metadata, data

# Parse your file
metadata, calib_data = parse_vicon_calibration_file(
    'CVIP\\Project\\data\\C3D\\Data_Run_Walk\\Data_Run_Walk\\AJ026\\Session1\\Calibration\\Calibration.csv'
)




=== PARSING VICON CALIBRATION FILE ===
Metadata:
  FrameNumber: 40.1
  FirstFrame: 1
  PointFrequency: 100
  AnalogFrequency: 100

Data shape: (403, 571)
First few columns: ['Time', 'LFHD', 'Unnamed: 2', 'Unnamed: 3', 'RFHD', 'Unnamed: 5', 'Unnamed: 6', 'LBHD', 'Unnamed: 8', 'Unnamed: 9', 'RBHD', 'Unnamed: 11', 'Unnamed: 12', 'LMAS', 'Unnamed: 14', 'Unnamed: 15', 'RMAS', 'Unnamed: 17', 'Unnamed: 18', 'GLAB']


In [18]:
print(calib_data.shape)
print(metadata)
print(calib_data.head())

(403, 571)
{'FrameNumber': '40.1', 'FirstFrame': '1', 'PointFrequency': '100', 'AnalogFrequency': '100'}
   Time     LFHD Unnamed: 2 Unnamed: 3     RFHD Unnamed: 5 Unnamed: 6  \
0     s       mm         mm         mm       mm         mm         mm   
1   NaN        X          Y          Z        X          Y          Z   
2     0  144.462    -42.247   1523.584  138.731    101.001   1531.458   
3  0.01  144.623     -42.03   1523.559  138.768     101.04   1531.491   
4  0.02  144.342     -42.21   1523.587  138.719     101.06   1531.426   

      LBHD Unnamed: 8 Unnamed: 9  ... Unnamed: 561 RThoraxAngles Unnamed: 563  \
0       mm         mm         mm  ...          deg           deg          deg   
1        X          Y          Z  ...            Z             X            Y   
2  272.368    -29.649   1485.246  ...        -2.26         5.166       -3.373   
3  272.366    -29.598   1485.245  ...       -2.262         5.159       -3.381   
4  272.343    -29.606    1485.25  ...       -2.265 

Extract marker positions and joint centers from Vicon data

In [12]:
def extract_vicon_marker_joint_info(calib_data):
    """
    Extract marker positions and joint centers from Vicon data
    """
    
    # The first row after header contains the marker and joint names
    # Second row contains units (mm, deg, etc.)
    # Third row contains coordinate labels (X, Y, Z)
    
    print("=== EXTRACTING VICON MARKER AND JOINT DATA ===")
    
    # Get all column names
    all_columns = list(calib_data.columns)
    
    # Separate different data types
    marker_data = {}
    joint_center_data = {}
    angle_data = {}
    imu_data = {}
    segment_data = {}
    
    current_category = None
    current_marker = None
    
    for i, col in enumerate(all_columns):
        if col == 'Time':
            continue
            
        # Check if this is a new marker/joint category
        if pd.notna(calib_data.iloc[0, i]) and calib_data.iloc[0, i] != '':
            current_marker = calib_data.iloc[0, i]
            current_category = classify_data_type(current_marker)
        
        # Get the coordinate type from row 2
        coord_type = calib_data.iloc[2, i] if i < len(calib_data.iloc[2]) else 'X'
        
        # Store data based on category
        if current_category == 'marker' and current_marker:
            if current_marker not in marker_data:
                marker_data[current_marker] = {}
            marker_data[current_marker][coord_type] = calib_data.iloc[3:, i].values
        
        elif current_category == 'joint_center' and current_marker:
            if current_marker not in joint_center_data:
                joint_center_data[current_marker] = {}
            joint_center_data[current_marker][coord_type] = calib_data.iloc[3:, i].values
        
        elif current_category == 'angle' and current_marker:
            if current_marker not in angle_data:
                angle_data[current_marker] = {}
            angle_data[current_marker][coord_type] = calib_data.iloc[3:, i].values
    
    print(f"Found {len(marker_data)} markers")
    print(f"Found {len(joint_center_data)} joint centers") 
    print(f"Found {len(angle_data)} angle measurements")
    
    return marker_data, joint_center_data, angle_data

def classify_data_type(name):
    """Classify the type of data based on name"""
    name = str(name).upper()
    
    # Marker names (anatomical landmarks)
    marker_keywords = ['LFHD', 'RFHD', 'LBHD', 'RBHD', 'LMAS', 'RMAS', 'C7', 'T2', 'T10', 
                      'CLAV', 'STRN', 'RBAK', 'LSHO', 'RSHO', 'LUPA', 'RUPA', 'LELB', 'RELB',
                      'LFRM', 'RFRM', 'LWRA', 'RWRA', 'LWRB', 'RWRB', 'LFIN', 'RFIN',
                      'LASI', 'RASI', 'LPSI', 'RPSI', 'LTHI', 'RTHI', 'LKNE', 'RKNE',
                      'LTIB', 'RTIB', 'LANK', 'RANK', 'LHEE', 'RHEE', 'LTOE', 'RTOE']
    
    # Joint centers (calculated)
    joint_center_keywords = ['LHJC', 'RHJC', 'LKJC', 'RKJC', 'LAJC', 'RAJC', 'LFJC', 'RFJC']
    
    # Angles
    angle_keywords = ['HipAngles', 'KneeAngles', 'AnkleAngles', 'SpineAngles', 'ShoulderAngles']
    
    if any(keyword in name for keyword in marker_keywords):
        return 'marker'
    elif any(keyword in name for keyword in joint_center_keywords):
        return 'joint_center'
    elif any(keyword in name for keyword in angle_keywords):
        return 'angle'
    elif 'IMU' in name:
        return 'imu'
    else:
        return 'segment'

In [19]:
def create_complete_anatomical_mapping():
    """
    Create comprehensive mapping from Vicon marker names to anatomical locations
    """
    anatomical_mapping = {
        # Head Markers
        'LFHD': 'Left_Front_Head',
        'RFHD': 'Right_Front_Head', 
        'LBHD': 'Left_Back_Head',
        'RBHD': 'Right_Back_Head',
        'LMAS': 'Left_Mastoid',
        'RMAS': 'Right_Mastoid',
        'GLAB': 'Glabella',
        
        # Trunk and Spine Markers
        'C7': 'C7_Vertebra',
        'T2': 'T2_Vertebra',
        'T10': 'T10_Vertebra',
        'CLAV': 'Clavicle',
        'STRN': 'Sternum',
        'RBAK': 'Right_Back',
        
        # Upper Body - Left Side
        'LSHO': 'Left_Shoulder',
        'LUPA': 'Left_Upper_Arm',
        'LELB': 'Left_Elbow',
        'LFRM': 'Left_Forearm',
        'LWRA': 'Left_Wrist_A',
        'LWRB': 'Left_Wrist_B',
        'LFIN': 'Left_Finger',
        
        # Upper Body - Right Side  
        'RSHO': 'Right_Shoulder',
        'RUPA': 'Right_Upper_Arm',
        'RELB': 'Right_Elbow',
        'RFRM': 'Right_Forearm',
        'RWRA': 'Right_Wrist_A',
        'RWRB': 'Right_Wrist_B',
        'RFIN': 'Right_Finger',
        
        # Pelvis Markers
        'LASI': 'Left_ASIS',
        'RASI': 'Right_ASIS',
        'LPSI': 'Left_PSIS',
        'RPSI': 'Right_PSIS',
        
        # Lower Body - Left Side
        'LTHI': 'Left_Thigh',
        'LTHAP': 'Left_Thigh_Anterior_Proximal',
        'LTHAD': 'Left_Thigh_Anterior_Distal',
        'LKNE': 'Left_Knee',
        'LKNM': 'Left_Knee_Medial',
        'LTIB': 'Left_Shank',
        'LTIAP': 'Left_Shank_Anterior_Proximal',
        'LTIAD': 'Left_Shank_Anterior_Distal',
        'LANK': 'Left_Ankle',
        'LMED': 'Left_Ankle_Medial',
        'LHEE': 'Left_Heel',
        'LTOE': 'Left_Toe',
        'LFMH': 'Left_Foot_Metatarsal_Head',
        'LSMH': 'Left_Foot_Second_Metatarsal_Head',
        'LVMH': 'Left_Foot_Fifth_Metatarsal_Head',
        'LFootOff': 'Left_Foot_Off_Trigger',
        
        # Lower Body - Right Side
        'RTHI': 'Right_Thigh',
        'RTHAP': 'Right_Thigh_Anterior_Proximal',
        'RTHAD': 'Right_Thigh_Anterior_Distal',
        'RKNE': 'Right_Knee',
        'RKNM': 'Right_Knee_Medial',
        'RTIB': 'Right_Shank',
        'RTIAP': 'Right_Shank_Anterior_Proximal',
        'RTIAD': 'Right_Shank_Anterior_Distal',
        'RANK': 'Right_Ankle',
        'RMED': 'Right_Ankle_Medial',
        'RHEE': 'Right_Heel',
        'RTOE': 'Right_Toe',
        'RFMH': 'Right_Foot_Metatarsal_Head',
        'RSMH': 'Right_Foot_Second_Metatarsal_Head',
        'RVMH': 'Right_Foot_Fifth_Metatarsal_Head',
        'RFootOff': 'Right_Foot_Off_Trigger',
        
        # IMU Orientation Data - Trunk
        'TrunkIMUO': 'Trunk_IMU_Orientation',
        'TrunkIMUP': 'Trunk_IMU_Position',
        'TrunkIMUL': 'Trunk_IMU_Linear',
        
        # IMU Orientation Data - Pelvis
        'PelvisIMUO': 'Pelvis_IMU_Orientation',
        'PelvisIMUP': 'Pelvis_IMU_Position',
        'PelvisIMUL': 'Pelvis_IMU_Linear',
        
        # IMU Orientation Data - Left Leg
        'LFemurIMUO': 'Left_Femur_IMU_Orientation',
        'LFemurIMUP': 'Left_Femur_IMU_Position',
        'LFemurIMUL': 'Left_Femur_IMU_Linear',
        'LTibiaIMUO': 'Left_Tibia_IMU_Orientation',
        'LTibiaIMUP': 'Left_Tibia_IMU_Position',
        'LTibiaIMUL': 'Left_Tibia_IMU_Linear',
        
        # IMU Orientation Data - Right Leg
        'RFemurIMUO': 'Right_Femur_IMU_Orientation',
        'RFemurIMUP': 'Right_Femur_IMU_Position',
        'RFemurIMUL': 'Right_Femur_IMU_Linear',
        'RTibiaIMUO': 'Right_Tibia_IMU_Orientation',
        'RTibiaIMUP': 'Right_Tibia_IMU_Position',
        'RTibiaIMUL': 'Right_Tibia_IMU_Linear',
        
        # Segment Parameters - Pelvis
        'PELVISO': 'Pelvis_Orientation',
        'PELVISA': 'Pelvis_Angle',
        'PELVISL': 'Pelvis_Location',
        'PELVISP': 'Pelvis_Position',
        
        # Segment Parameters - Left Leg
        'LFEMURO': 'Left_Femur_Orientation',
        'LFEMURA': 'Left_Femur_Angle',
        'LFEMURL': 'Left_Femur_Location',
        'LFEMURP': 'Left_Femur_Position',
        'LTIBIAO': 'Left_Tibia_Orientation',
        'LTIBIAA': 'Left_Tibia_Angle',
        'LTIBIAL': 'Left_Tibia_Location',
        'LTIBIAP': 'Left_Tibia_Position',
        'LFOOTO': 'Left_Foot_Orientation',
        'LFOOTA': 'Left_Foot_Angle',
        'LFOOTL': 'Left_Foot_Location',
        'LFOOTP': 'Left_Foot_Position',
        'LTOESO': 'Left_Toes_Orientation',
        'LTOESA': 'Left_Toes_Angle',
        'LTOESL': 'Left_Toes_Location',
        'LTOESP': 'Left_Toes_Position',
        
        # Segment Parameters - Right Leg
        'RFEMURO': 'Right_Femur_Orientation',
        'RFEMURA': 'Right_Femur_Angle',
        'RFEMURL': 'Right_Femur_Location',
        'RFEMURP': 'Right_Femur_Position',
        'RTIBIAO': 'Right_Tibia_Orientation',
        'RTIBIAA': 'Right_Tibia_Angle',
        'RTIBIAL': 'Right_Tibia_Location',
        'RTIBIAP': 'Right_Tibia_Position',
        'RFOOTO': 'Right_Foot_Orientation',
        'RFOOTA': 'Right_Foot_Angle',
        'RFOOTL': 'Right_Foot_Location',
        'RFOOTP': 'Right_Foot_Position',
        'RTOESO': 'Right_Toes_Orientation',
        'RTOESA': 'Right_Toes_Angle',
        'RTOESL': 'Right_Toes_Location',
        'RTOESP': 'Right_Toes_Position',
        
        # Segment Parameters - Upper Body
        'THORAXO': 'Thorax_Orientation',
        'THORAXA': 'Thorax_Angle',
        'THORAXL': 'Thorax_Location',
        'THORAXP': 'Thorax_Position',
        'LUPPERARMO': 'Left_Upper_Arm_Orientation',
        'LUPPERARMA': 'Left_Upper_Arm_Angle',
        'LUPPERARML': 'Left_Upper_Arm_Location',
        'LUPPERARMP': 'Left_Upper_Arm_Position',
        'LFOREARMO': 'Left_Forearm_Orientation',
        'LFOREARMA': 'Left_Forearm_Angle',
        'LFOREARML': 'Left_Forearm_Location',
        'LFOREARMP': 'Left_Forearm_Position',
        'LHANDO': 'Left_Hand_Orientation',
        'LHANDA': 'Left_Hand_Angle',
        'LHANDL': 'Left_Hand_Location',
        'LHANDP': 'Left_Hand_Position',
        'RUPPERARMO': 'Right_Upper_Arm_Orientation',
        'RUPPERARMA': 'Right_Upper_Arm_Angle',
        'RUPPERARML': 'Right_Upper_Arm_Location',
        'RUPPERARMP': 'Right_Upper_Arm_Position',
        'RFOREARMO': 'Right_Forearm_Orientation',
        'RFOREARMA': 'Right_Forearm_Angle',
        'RFOREARML': 'Right_Forearm_Location',
        'RFOREARMP': 'Right_Forearm_Position',
        'RHANDO': 'Right_Hand_Orientation',
        'RHANDA': 'Right_Hand_Angle',
        'RHANDL': 'Right_Hand_Location',
        'RHANDP': 'Right_Hand_Position',
        'HEADO': 'Head_Orientation',
        'HEADA': 'Head_Angle',
        'HEADL': 'Head_Location',
        'HEADP': 'Head_Position',
        
        # Joint Centers
        'LHJC': 'Left_Hip_Joint_Center',
        'RHJC': 'Right_Hip_Joint_Center',
        'LKJC': 'Left_Knee_Joint_Center',
        'RKJC': 'Right_Knee_Joint_Center',
        'LAJC': 'Left_Ankle_Joint_Center',
        'RAJC': 'Right_Ankle_Joint_Center',
        'LFJC': 'Left_Foot_Joint_Center',
        'RFJC': 'Right_Foot_Joint_Center',
        'LSJC': 'Left_Shoulder_Joint_Center',
        'RSJC': 'Right_Shoulder_Joint_Center',
        'LEJC': 'Left_Elbow_Joint_Center',
        'REJC': 'Right_Elbow_Joint_Center',
        'LHO': 'Left_Hand_Offset',
        'RHO': 'Right_Hand_Offset',
        
        # Center of Mass
        'CentreOfMass': 'Center_of_Mass',
        
        # Joint Angles - Left Side
        'LHipAngles': 'Left_Hip_Angles',
        'LKneeAngles': 'Left_Knee_Angles',
        'LAnkleAngles': 'Left_Ankle_Angles',
        'LForeFootAngles': 'Left_ForeFoot_Angles',
        
        # Joint Angles - Right Side
        'RHipAngles': 'Right_Hip_Angles',
        'RKneeAngles': 'Right_Knee_Angles',
        'RAnkleAngles': 'Right_Ankle_Angles',
        'RForeFootAngles': 'Right_ForeFoot_Angles',
        
        # Spine and Neck Angles
        'LSpineAngles': 'Left_Spine_Angles',
        'RSpineAngles': 'Right_Spine_Angles',
        'LNeckAngles': 'Left_Neck_Angles',
        'RNeckAngles': 'Right_Neck_Angles',
        
        # Upper Body Angles - Left Side
        'LShoulderAngles': 'Left_Shoulder_Angles',
        'LElbowAngles': 'Left_Elbow_Angles',
        'LWristAngles': 'Left_Wrist_Angles',
        
        # Upper Body Angles - Right Side
        'RShoulderAngles': 'Right_Shoulder_Angles',
        'RElbowAngles': 'Right_Elbow_Angles',
        'RWristAngles': 'Right_Wrist_Angles',
        
        # Foot and Pelvis Angles
        'LFootProgressAngles': 'Left_Foot_Progress_Angles',
        'RFootProgressAngles': 'Right_Foot_Progress_Angles',
        'LPelvisAngles': 'Left_Pelvis_Angles',
        'RPelvisAngles': 'Right_Pelvis_Angles',
        
        # Thorax and Head Angles
        'LThoraxAngles': 'Left_Thorax_Angles',
        'RThoraxAngles': 'Right_Thorax_Angles',
        'LHeadAngles': 'Left_Head_Angles',
        'RHeadAngles': 'Right_Head_Angles'
    }
    
    return anatomical_mapping

Build Skeleton from Calibration Data

In [ ]:
def build_skeleton_from_calibration(calib_data, frame=0):
    """
    Build anatomical skeleton from calibration data
    """
    
    # Use the first frame of calibration (static pose)
    frame_data = calib_data.iloc[3 + frame]  # Data starts at row 3
    
    print(frame_data.shape)

    anatomical_mapping = create_complete_anatomical_mapping()
    skeleton = {}
    
    # Extract marker positions
    current_marker = None
    for i, col_name in enumerate(calib_data.columns):
        if col_name == 'Time':
            continue
            
        # Check if this column contains a marker name
        marker_name = calib_data.iloc[0, i]
        if pd.notna(marker_name) and marker_name in anatomical_mapping:
            current_marker = marker_name
            skeleton[current_marker] = {'anatomical_name': anatomical_mapping[current_marker]}
        
        # Extract coordinates
        if current_marker and pd.notna(frame_data.iloc[i]):
            coord_type = calib_data.iloc[2, i]  # X, Y, or Z
            if coord_type in ['X', 'Y', 'Z']:
                if 'position' not in skeleton[current_marker]:
                    skeleton[current_marker]['position'] = {}
                skeleton[current_marker]['position'][coord_type] = frame_data.iloc[i]
    
    # Convert to numpy arrays
    for marker in skeleton:
        if 'position' in skeleton[marker]:
            pos = skeleton[marker]['position']
            skeleton[marker]['position_array'] = np.array([pos.get('X', 0), pos.get('Y', 0), pos.get('Z', 0)])
    
    print(f"Built skeleton with {len(skeleton)} anatomical markers")
    return skeleton

def visualize_calibration_skeleton(skeleton):
    """
    Visualize the skeleton from calibration data
    """
    fig = plt.figure(figsize=(15, 10))
    
    # 3D plot
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot markers
    colors = {'Left': 'red', 'Right': 'blue', 'Other': 'green'}
    
    for marker_name, marker_data in skeleton.items():
        if 'position_array' in marker_data:
            pos = marker_data['position_array']
            anatomical_name = marker_data['anatomical_name']
            
            # Determine color based on side
            if 'Left' in anatomical_name:
                color = colors['Left']
                marker = 'o'
            elif 'Right' in anatomical_name:
                color = colors['Right'] 
                marker = 'o'
            else:
                color = colors['Other']
                marker = 's'
            
            ax.scatter(pos[0], pos[1], pos[2], c=color, marker=marker, s=50, label=anatomical_name)
            ax.text(pos[0], pos[1], pos[2], anatomical_name, fontsize=8)
    
    # Draw basic connections
    connections = [
        # Head
        ('LFHD', 'RFHD'), ('LBHD', 'RBHD'), ('LFHD', 'LBHD'), ('RFHD', 'RBHD'),
        
        # Trunk
        ('C7', 'T10'), ('T10', 'LASI'), ('T10', 'RASI'),
        
        # Left Arm
        ('LSHO', 'LELB'), ('LELB', 'LWRA'),
        
        # Right Arm
        ('RSHO', 'RELB'), ('RELB', 'RWRA'),
        
        # Left Leg
        ('LASI', 'LKNE'), ('LKNE', 'LANK'), ('LANK', 'LTOE'),
        
        # Right Leg
        ('RASI', 'RKNE'), ('RKNE', 'RANK'), ('RANK', 'RTOE')
    ]
    
    for start, end in connections:
        if start in skeleton and end in skeleton:
            start_pos = skeleton[start]['position_array']
            end_pos = skeleton[end]['position_array']
            ax.plot([start_pos[0], end_pos[0]], 
                   [start_pos[1], end_pos[1]], 
                   [start_pos[2], end_pos[2]], 'k-', linewidth=2)
    
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_zlabel('Z (mm)')
    ax.set_title('Calibration Skeleton - Static Pose')
    
    # Create legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    
    plt.tight_layout()
    plt.show()

# Build and visualize skeleton
skeleton = build_skeleton_from_calibration(calib_data, frame=0)
visualize_calibration_skeleton(skeleton)

In [22]:
def build_complete_skeleton_from_calibration(calib_data, frame=0):
    """
    Build complete anatomical skeleton from calibration data with T-pose
    
    Parameters:
    - calib_data: DataFrame from calibration file
    - frame: Which frame to use (default 0 for static pose)
    """
    
    # Skip header rows and get the actual data
    # Typically Vicon data has 3 header rows: Marker names, something, coordinate labels
    data_start_row = 3  # Adjust based on your file structure
    
    if frame + data_start_row >= len(calib_data):
        frame = 0  # Use first frame if requested frame doesn't exist
    
    frame_data = calib_data.iloc[data_start_row + frame]
    
    print(f"Frame data shape: {frame_data.shape}")
    print(f"Number of columns: {len(calib_data.columns)}")
    
    anatomical_mapping = create_complete_anatomical_mapping()
    skeleton = {}
    
    # Vicon typically stores data in pattern: Marker1_X, Marker1_Y, Marker1_Z, Marker2_X, Marker2_Y, Marker2_Z, etc.
    current_marker = None
    marker_coords = {}
    
    for i, col_name in enumerate(calib_data.columns):
        if col_name == 'Time':
            continue
            
        # Get marker name from header (usually row 0)
        if i < len(calib_data.columns):
            potential_marker = calib_data.iloc[0, i]
            if pd.notna(potential_marker) and potential_marker in anatomical_mapping:
                # If we were collecting coordinates for previous marker, save them
                if current_marker and marker_coords:
                    skeleton[current_marker] = {
                        'anatomical_name': anatomical_mapping[current_marker],
                        'position': marker_coords.copy(),
                        'position_array': np.array([marker_coords.get('X', 0), 
                                                  marker_coords.get('Y', 0), 
                                                  marker_coords.get('Z', 0)])
                    }
                
                # Start new marker
                current_marker = potential_marker
                marker_coords = {}
            
            # Get coordinate type (X, Y, Z) - usually row 2
            coord_type = calib_data.iloc[2, i] if 2 < len(calib_data) else None
            
            # Store coordinate if we have a current marker and valid data
            if current_marker and coord_type in ['X', 'Y', 'Z'] and pd.notna(frame_data.iloc[i]):
                marker_coords[coord_type] = frame_data.iloc[i]
    
    # Don't forget the last marker
    if current_marker and marker_coords:
        skeleton[current_marker] = {
            'anatomical_name': anatomical_mapping[current_marker],
            'position': marker_coords.copy(),
            'position_array': np.array([marker_coords.get('X', 0), 
                                      marker_coords.get('Y', 0), 
                                      marker_coords.get('Z', 0)])
        }
    
    print(f"Built skeleton with {len(skeleton)} anatomical markers")
    
    # Print some diagnostics
    markers_by_region = {}
    for marker, data in skeleton.items():
        region = data['anatomical_name'].split('_')[0]  # Get first part of name
        if region not in markers_by_region:
            markers_by_region[region] = []
        markers_by_region[region].append(marker)
    
    print("\nMarkers by body region:")
    for region, markers in markers_by_region.items():
        print(f"  {region}: {len(markers)} markers")
    
    return skeleton

def visualize_skeleton_structure(skeleton):
    """
    Print the structure of the built skeleton for verification
    """
    print("\n=== SKELETON STRUCTURE ===")
    
    regions = {
        'Head': [], 'Trunk': [], 'Pelvis': [],
        'Left_Upper': [], 'Left_Lower': [],
        'Right_Upper': [], 'Right_Lower': []
    }
    
    for marker, data in skeleton.items():
        anatomical_name = data['anatomical_name']
        pos = data['position_array']
        
        # Categorize by region
        if any(prefix in anatomical_name for prefix in ['Head', 'Mastoid', 'Glabella']):
            regions['Head'].append(marker)
        elif any(prefix in anatomical_name for prefix in ['C7', 'T2', 'T10', 'Clavicle', 'Sternum', 'Back', 'Thorax']):
            regions['Trunk'].append(marker)
        elif any(prefix in anatomical_name for prefix in ['ASIS', 'PSIS', 'Pelvis']):
            regions['Pelvis'].append(marker)
        elif anatomical_name.startswith('Left_') and any(part in anatomical_name for part in ['Shoulder', 'Arm', 'Elbow', 'Wrist', 'Hand', 'Finger']):
            regions['Left_Upper'].append(marker)
        elif anatomical_name.startswith('Left_') and any(part in anatomical_name for part in ['Thigh', 'Knee', 'Shank', 'Ankle', 'Foot', 'Heel', 'Toe']):
            regions['Left_Lower'].append(marker)
        elif anatomical_name.startswith('Right_') and any(part in anatomical_name for part in ['Shoulder', 'Arm', 'Elbow', 'Wrist', 'Hand', 'Finger']):
            regions['Right_Upper'].append(marker)
        elif anatomical_name.startswith('Right_') and any(part in anatomical_name for part in ['Thigh', 'Knee', 'Shank', 'Ankle', 'Foot', 'Heel', 'Toe']):
            regions['Right_Lower'].append(marker)
    
    for region, markers in regions.items():
        print(f"\n{region} ({len(markers)} markers):")
        for marker in markers:
            pos = skeleton[marker]['position_array']
            print(f"  {marker} -> {skeleton[marker]['anatomical_name']}")
            print(f"    Position: [{pos[0]:.1f}, {pos[1]:.1f}, {pos[2]:.1f}]")

def calculate_skeleton_dimensions(skeleton):
    """
    Calculate basic dimensions of the skeleton for verification
    """
    print("\n=== SKELETON DIMENSIONS ===")
    
    # Get key landmarks
    landmarks = {}
    for marker, data in skeleton.items():
        anatomical_name = data['anatomical_name']
        if 'Left_ASIS' in anatomical_name:
            landmarks['left_hip'] = data['position_array']
        elif 'Right_ASIS' in anatomical_name:
            landmarks['right_hip'] = data['position_array']
        elif 'Left_Shoulder' in anatomical_name:
            landmarks['left_shoulder'] = data['position_array']
        elif 'Right_Shoulder' in anatomical_name:
            landmarks['right_shoulder'] = data['position_array']
        elif 'Left_Heel' in anatomical_name:
            landmarks['left_heel'] = data['position_array']
        elif 'Right_Heel' in anatomical_name:
            landmarks['right_heel'] = data['position_array']
        elif 'Left_Toe' in anatomical_name:
            landmarks['left_toe'] = data['position_array']
        elif 'Right_Toe' in anatomical_name:
            landmarks['right_toe'] = data['position_array']
    
    # Calculate dimensions
    if 'left_shoulder' in landmarks and 'right_shoulder' in landmarks:
        shoulder_width = np.linalg.norm(landmarks['left_shoulder'] - landmarks['right_shoulder'])
        print(f"Shoulder width: {shoulder_width:.1f} mm")
    
    if 'left_hip' in landmarks and 'right_hip' in landmarks:
        hip_width = np.linalg.norm(landmarks['left_hip'] - landmarks['right_hip'])
        print(f"Hip width: {hip_width:.1f} mm")
    
    if 'left_heel' in landmarks and 'left_toe' in landmarks:
        foot_length = np.linalg.norm(landmarks['left_heel'] - landmarks['left_toe'])
        print(f"Foot length: {foot_length:.1f} mm")

# Usage example:
def process_calibration_file(calib_data):
    """
    Complete processing of calibration file
    """
    # Read calibration data
    # calib_data = pd.read_csv(calib_file_path)
    
    print("=== CALIBRATION DATA INFO ===")
    print(f"Data shape: {calib_data.shape}")
    print(f"Columns: {len(calib_data.columns)}")
    print("\nFirst few column names:")
    for i, col in enumerate(calib_data.columns[:10]):
        print(f"  {i}: {col}")
    
    print("\nFirst few rows of header:")
    for i in range(min(5, len(calib_data))):
        print(f"Row {i}: {calib_data.iloc[i, :5].tolist()}...")
    
    # Build skeleton
    skeleton = build_complete_skeleton_from_calibration(calib_data)
    
    # Visualize structure
    visualize_skeleton_structure(skeleton)
    
    # Calculate dimensions
    calculate_skeleton_dimensions(skeleton)
    
    return skeleton

In [23]:
# skeleton = process_calibration_file('CVIP\\Project\\data\\C3D\\Data_Run_Walk\\Data_Run_Walk\\AJ026\\Session1\\Calibration\\Calibration.csv')
skeleton = process_calibration_file(calib_data)

=== CALIBRATION DATA INFO ===
Data shape: (403, 571)
Columns: 571

First few column names:
  0: Time
  1: LFHD
  2: Unnamed: 2
  3: Unnamed: 3
  4: RFHD
  5: Unnamed: 5
  6: Unnamed: 6
  7: LBHD
  8: Unnamed: 8
  9: Unnamed: 9

First few rows of header:
Row 0: ['s', 'mm', 'mm', 'mm', 'mm']...
Row 1: [nan, 'X', 'Y', 'Z', 'X']...
Row 2: ['0', '144.462', '-42.247', '1523.584', '138.731']...
Row 3: ['0.01', '144.623', '-42.03', '1523.559', '138.768']...
Row 4: ['0.02', '144.342', '-42.21', '1523.587', '138.719']...
Frame data shape: (571,)
Number of columns: 571
Built skeleton with 0 anatomical markers

Markers by body region:

=== SKELETON STRUCTURE ===

Head (0 markers):

Trunk (0 markers):

Pelvis (0 markers):

Left_Upper (0 markers):

Left_Lower (0 markers):

Right_Upper (0 markers):

Right_Lower (0 markers):

=== SKELETON DIMENSIONS ===


In [27]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

def visualize_skeleton_3d(skeleton, title="3D Skeleton Visualization"):
    """
    Visualize the complete skeleton in 3D with interactive plot
    """
    # Extract positions and create groups for different marker types
    markers_3d = []
    marker_names = []
    marker_types = []
    colors = []
    sizes = []
    
    # Define color scheme
    color_scheme = {
        'head': 'red',
        'trunk': 'blue', 
        'pelvis': 'purple',
        'left_upper': 'green',
        'left_lower': 'lightgreen',
        'right_upper': 'orange',
        'right_lower': 'yellow',
        'joint_center': 'black',
        'imu': 'gray',
        'other': 'pink'
    }
    
    # Categorize markers and collect data
    for marker, data in skeleton.items():
        if 'position_array' in data and not np.all(data['position_array'] == 0):
            pos = data['position_array']
            markers_3d.append(pos)
            marker_names.append(f"{marker}\n({data['anatomical_name']})")
            
            anatomical_name = data['anatomical_name'].lower()
            
            # Categorize by type
            if any(term in anatomical_name for term in ['head', 'mastoid', 'glabella']):
                marker_type = 'head'
                size = 8
            elif any(term in anatomical_name for term in ['c7', 't2', 't10', 'clavicle', 'sternum', 'back', 'thorax', 'spine']):
                marker_type = 'trunk'
                size = 7
            elif any(term in anatomical_name for term in ['asis', 'psis', 'pelvis']):
                marker_type = 'pelvis'
                size = 7
            elif any(term in anatomical_name for term in ['shoulder', 'arm', 'elbow', 'wrist', 'hand', 'finger']):
                if anatomical_name.startswith('left'):
                    marker_type = 'left_upper'
                else:
                    marker_type = 'right_upper'
                size = 6
            elif any(term in anatomical_name for term in ['thigh', 'knee', 'shank', 'ankle', 'foot', 'heel', 'toe']):
                if anatomical_name.startswith('left'):
                    marker_type = 'left_lower'
                else:
                    marker_type = 'right_lower'
                size = 6
            elif any(term in anatomical_name for term in ['joint', 'center', 'jhc', 'kjc', 'ajc', 'fjc', 'sjc', 'ejc']):
                marker_type = 'joint_center'
                size = 5
            elif any(term in anatomical_name for term in ['imu']):
                marker_type = 'imu'
                size = 4
            else:
                marker_type = 'other'
                size = 4
                
            marker_types.append(marker_type)
            colors.append(color_scheme[marker_type])
            sizes.append(size)
    
    if not markers_3d:
        print("No valid marker positions found!")
        return
    
    markers_3d = np.array(markers_3d)
    
    # Create the figure
    fig = go.Figure()
    
    # Add markers
    fig.add_trace(go.Scatter3d(
        x=markers_3d[:, 0],
        y=markers_3d[:, 1], 
        z=markers_3d[:, 2],
        mode='markers+text',
        marker=dict(
            size=sizes,
            color=colors,
            opacity=0.8,
            line=dict(width=2, color='darkgray')
        ),
        text=marker_names,
        textposition="top center",
        name='Anatomical Markers',
        hovertemplate=(
            "<b>%{text}</b><br>" +
            "X: %{x:.1f}<br>" +
            "Y: %{y:.1f}<br>" +
            "Z: %{z:.1f}<br>" +
            "<extra></extra>"
        )
    ))
    
    # Add connecting lines for major body segments
    add_skeleton_connections(fig, skeleton)
    
    # Set layout
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            font=dict(size=16)
        ),
        scene=dict(
            xaxis_title='X (mm)',
            yaxis_title='Y (mm)', 
            zaxis_title='Z (mm)',
            aspectmode='data',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        width=1000,
        height=800,
        showlegend=True
    )
    
    # Add legend for marker types
    for marker_type, color in color_scheme.items():
        fig.add_trace(go.Scatter3d(
            x=[None], y=[None], z=[None],
            mode='markers',
            marker=dict(size=10, color=color),
            name=marker_type.replace('_', ' ').title(),
            showlegend=True
        ))
    
    fig.show()
    
    return fig

def add_skeleton_connections(fig, skeleton):
    """
    Add lines connecting major body segments to create skeleton structure
    """
    connections = [
        # Head
        ['LFHD', 'RFHD'], ['RFHD', 'RBHD'], ['RBHD', 'LBHD'], ['LBHD', 'LFHD'],
        ['LFHD', 'GLAB'], ['RFHD', 'GLAB'],
        
        # Trunk
        ['C7', 'T2'], ['T2', 'T10'], ['T10', 'STRN'], ['STRN', 'CLAV'],
        ['CLAV', 'LSHO'], ['CLAV', 'RSHO'],
        
        # Left Arm
        ['LSHO', 'LUPA'], ['LUPA', 'LELB'], ['LELB', 'LFRM'], 
        ['LFRM', 'LWRA'], ['LWRA', 'LWRB'], ['LWRB', 'LFIN'],
        
        # Right Arm  
        ['RSHO', 'RUPA'], ['RUPA', 'RELB'], ['RELB', 'RFRM'],
        ['RFRM', 'RWRA'], ['RWRA', 'RWRB'], ['RWRB', 'RFIN'],
        
        # Pelvis
        ['LASI', 'RASI'], ['LASI', 'LPSI'], ['RASI', 'RPSI'], ['LPSI', 'RPSI'],
        
        # Left Leg
        ['LASI', 'LTHI'], ['LTHI', 'LKNE'], ['LKNE', 'LTIB'], ['LTIB', 'LANK'],
        ['LANK', 'LHEE'], ['LANK', 'LTOE'], ['LHEE', 'LFMH'], ['LFMH', 'LTOE'],
        
        # Right Leg
        ['RASI', 'RTHI'], ['RTHI', 'RKNE'], ['RKNE', 'RTIB'], ['RTIB', 'RANK'],
        ['RANK', 'RHEE'], ['RANK', 'RTOE'], ['RHEE', 'RFMH'], ['RFMH', 'RTOE'],
    ]
    
    for connection in connections:
        marker1, marker2 = connection
        if marker1 in skeleton and marker2 in skeleton:
            if 'position_array' in skeleton[marker1] and 'position_array' in skeleton[marker2]:
                pos1 = skeleton[marker1]['position_array']
                pos2 = skeleton[marker2]['position_array']
                
                # Only draw if positions are valid (not all zeros)
                if not (np.all(pos1 == 0) or np.all(pos2 == 0)):
                    fig.add_trace(go.Scatter3d(
                        x=[pos1[0], pos2[0]],
                        y=[pos1[1], pos2[1]], 
                        z=[pos1[2], pos2[2]],
                        mode='lines',
                        line=dict(color='black', width=3),
                        showlegend=False,
                        hoverinfo='skip'
                    ))

def visualize_skeleton_subplots(skeleton):
    """
    Create multiple views of the skeleton (front, side, top)
    """
    from plotly.subplots import make_subplots
    
    # Extract positions
    markers_3d = []
    marker_names = []
    colors = []
    
    for marker, data in skeleton.items():
        if 'position_array' in data and not np.all(data['position_array'] == 0):
            pos = data['position_array']
            markers_3d.append(pos)
            marker_names.append(marker)
            
            # Simple coloring by height for subplots
            colors.append(pos[2])  # Use Z-coordinate for color
    
    if not markers_3d:
        return
    
    markers_3d = np.array(markers_3d)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'is_3d': True}, {'is_3d': True}],
               [{'is_3d': True}, {'is_3d': False}]],
        subplot_titles=('Front View (X-Z)', 'Side View (Y-Z)', 'Top View (X-Y)', 'Marker List'),
        print_grid=False
    )
    
    # Front view (X-Z)
    fig.add_trace(go.Scatter3d(
        x=markers_3d[:, 0], y=np.zeros_like(markers_3d[:, 1]), z=markers_3d[:, 2],
        mode='markers', marker=dict(size=5, color=colors, colorscale='Viridis'),
        name='Front View'
    ), 1, 1)
    
    # Side view (Y-Z)  
    fig.add_trace(go.Scatter3d(
        x=np.zeros_like(markers_3d[:, 0]), y=markers_3d[:, 1], z=markers_3d[:, 2],
        mode='markers', marker=dict(size=5, color=colors, colorscale='Viridis'),
        name='Side View'
    ), 1, 2)
    
    # Top view (X-Y)
    fig.add_trace(go.Scatter3d(
        x=markers_3d[:, 0], y=markers_3d[:, 1], z=np.zeros_like(markers_3d[:, 2]),
        mode='markers', marker=dict(size=5, color=colors, colorscale='Viridis'), 
        name='Top View'
    ), 2, 1)
    
    # Marker list table
    marker_table = []
    for marker, data in skeleton.items():
        if 'position_array' in data:
            pos = data['position_array']
            marker_table.append({
                'Marker': marker,
                'Anatomical Name': data['anatomical_name'],
                'X': f"{pos[0]:.1f}",
                'Y': f"{pos[1]:.1f}", 
                'Z': f"{pos[2]:.1f}"
            })
    
    fig.update_layout(
        height=800,
        title_text="Skeleton Multiple Views",
        showlegend=False
    )
    
    return fig

# Enhanced processing function with 3D visualization
def process_and_visualize_calibration(calib_data, frame=0):
    """
    Complete processing and 3D visualization of calibration data
    """
    # Read and process calibration data
    # calib_data = pd.read_csv(calib_file_path)
    skeleton = build_complete_skeleton_from_calibration(calib_data, frame)
    
    # Print basic info
    print(f"=== 3D SKELETON VISUALIZATION ===")
    print(f"Total markers with valid positions: {len([m for m in skeleton if 'position_array' in skeleton[m] and not np.all(skeleton[m]['position_array'] == 0)])}")
    
    # Create 3D visualization
    fig = visualize_skeleton_3d(skeleton, f"Skeleton from Calibration Data (Frame {frame})")
    
    # Create multiple views
    fig_views = visualize_skeleton_subplots(skeleton)
    if fig_views:
        fig_views.show()
    
    return skeleton, fig

In [28]:
# Process your calibration file and visualize
skeleton, fig = process_and_visualize_calibration(calib_data, frame=0)

# Or just visualize an existing skeleton
visualize_skeleton_3d(skeleton, "My Skeleton Visualization")

Frame data shape: (571,)
Number of columns: 571
Built skeleton with 0 anatomical markers

Markers by body region:
=== 3D SKELETON VISUALIZATION ===
Total markers with valid positions: 0
No valid marker positions found!
No valid marker positions found!


In [ ]:
def build_skeleton_from_vicon_format(calib_data, frame=0):
    """
    Build skeleton from Vicon file with this specific structure:
    Row 0: Marker names
    Row 1: Units (mm/deg)
    Row 2: Coordinate labels (X, Y, Z)
    Row 3+: Actual data values
    """
    print("=== BUILDING SKELETON FROM VICON FORMAT ===")
    
    anatomical_mapping = create_complete_anatomical_mapping()
    skeleton = {}
    
    # The data starts at row 3 (index 3)
    data_start_row = 3
    
    if data_start_row + frame >= len(calib_data):
        print(f"Warning: Frame {frame} doesn't exist, using frame 0")
        frame = 0
    
    frame_data = calib_data.iloc[data_start_row + frame]
    print(f"Using data from row {data_start_row + frame}")
    
    # Group columns by marker - each marker has 3 consecutive columns (X, Y, Z)
    current_marker = None
    marker_coords = {}
    
    for col_idx in range(len(calib_data.columns)):
        # Get marker name from row 0
        marker_name = calib_data.iloc[0, col_idx]
        
        if pd.notna(marker_name) and marker_name != 'Time' and marker_name != 'AnalogFret':
            # This is a marker column
            current_marker = str(marker_name).strip()
            
            # Reset coordinates for new marker
            if current_marker not in anatomical_mapping:
                # Skip markers we don't have in our mapping
                current_marker = None
                continue
        
        # Get coordinate type from row 2
        coord_type = calib_data.iloc[2, col_idx] if 2 < len(calib_data) else None
        
        # If we have a current marker and this is a coordinate column, store the value
        if current_marker and coord_type in ['X', 'Y', 'Z'] and pd.notna(frame_data.iloc[col_idx]):
            try:
                coord_value = float(frame_data.iloc[col_idx])
                if current_marker not in marker_coords:
                    marker_coords[current_marker] = {}
                marker_coords[current_marker][coord_type] = coord_value
                
                # If we have all 3 coordinates, add to skeleton
                if len(marker_coords[current_marker]) == 3:
                    skeleton[current_marker] = {
                        'anatomical_name': anatomical_mapping[current_marker],
                        'position': marker_coords[current_marker].copy(),
                        'position_array': np.array([
                            marker_coords[current_marker].get('X', 0),
                            marker_coords[current_marker].get('Y', 0), 
                            marker_coords[current_marker].get('Z', 0)
                        ])
                    }
                    print(f"Added {current_marker}: X={marker_coords[current_marker]['X']:.1f}, Y={marker_coords[current_marker]['Y']:.1f}, Z={marker_coords[current_marker]['Z']:.1f}")
                    marker_coords[current_marker] = {}  # Reset for next occurrence
            except (ValueError, TypeError):
                print(f"Invalid data for {current_marker} at column {col_idx}, skipping")
    
    print(f"Built skeleton with {len(skeleton)} anatomical markers")
    return skeleton

def build_skeleton_robust(calib_data, frame=0):
    """
    More robust version that handles the exact Vicon format
    """
    print("=== ROBUST SKELETON BUILDER ===")
    
    anatomical_mapping = create_complete_anatomical_mapping()
    print(anatomical_mapping)
    skeleton = {}
    
    data_start_row = 3
    if data_start_row + frame >= len(calib_data):
        frame = 0
    
    frame_data = calib_data.iloc[data_start_row + frame]
    
    # Create a mapping of marker to its three column indices
    marker_columns = {}
    
    for col_idx in range(len(calib_data.columns)):
        marker_name = calib_data.iloc[0, col_idx]
        
        if pd.notna(marker_name) and marker_name in anatomical_mapping:
            marker_name = str(marker_name).strip()
            if marker_name not in marker_columns:
                marker_columns[marker_name] = []
            marker_columns[marker_name].append(col_idx)
    
    print(f"Found {len(marker_columns)} unique markers in header")
    
    # For each marker, find its X, Y, Z coordinates
    for marker_name, col_indices in marker_columns.items():
        coords = {}
        
        for col_idx in col_indices:
            coord_type = calib_data.iloc[2, col_idx] if 2 < len(calib_data) else None
            
            if coord_type in ['X', 'Y', 'Z'] and pd.notna(frame_data.iloc[col_idx]):
                try:
                    value = float(frame_data.iloc[col_idx])
                    coords[coord_type] = value
                except (ValueError, TypeError):
                    coords[coord_type] = 0.0
        
        # Only add if we have all three coordinates
        if 'X' in coords and 'Y' in coords and 'Z' in coords:
            skeleton[marker_name] = {
                'anatomical_name': anatomical_mapping[marker_name],
                'position': coords,
                'position_array': np.array([coords['X'], coords['Y'], coords['Z']])
            }
    
    print(f"Successfully built skeleton with {len(skeleton)} markers")
    return skeleton

def analyze_vicon_structure(calib_data):
    """
    Analyze the exact structure of the Vicon file
    """
    print("=== VICON FILE STRUCTURE ANALYSIS ===")
    print(f"Total rows: {len(calib_data)}")
    print(f"Total columns: {len(calib_data.columns)}")
    
    # Analyze first few rows
    for row_idx in range(min(4, len(calib_data))):
        print(f"\n--- Row {row_idx} ---")
        markers_found = []
        
        for col_idx in range(min(20, len(calib_data.columns))):  # Check first 20 columns
            value = calib_data.iloc[row_idx, col_idx]
            if pd.notna(value):
                if row_idx == 0 and value != 'Time' and value != 'AnalogFret':
                    markers_found.append(str(value))
                print(f"Col {col_idx}: {repr(value)}")
        
        if row_idx == 0 and markers_found:
            print(f"Markers in row 0: {markers_found}")
    
    # Count how many markers we can find
    anatomical_mapping = create_complete_anatomical_mapping()
    found_markers = []
    
    for col_idx in range(len(calib_data.columns)):
        marker_name = calib_data.iloc[0, col_idx]
        if pd.notna(marker_name) and marker_name in anatomical_mapping:
            found_markers.append(marker_name)
    
    print(f"\nFound {len(found_markers)} known markers in file:")
    print(found_markers)

def process_vicon_calibration(calib_data, frame=0):
    """
    Complete processing for Vicon calibration files
    """
    print("=" * 60)
    print("PROCESSING VICON CALIBRATION FILE")
    print("=" * 60)
    
    try:
        # Read the file
        # calib_data = pd.read_csv(calib_file_path)
        
        # First analyze the structure
        analyze_vicon_structure(calib_data)
        
        # Try the robust builder
        skeleton = build_skeleton_robust(calib_data, frame)
        
        if len(skeleton) == 0:
            print("\n⚠️  No markers found with robust method, trying alternative...")
            skeleton = build_skeleton_from_vicon_format(calib_data, frame)
        
        if len(skeleton) == 0:
            print("\n❌ ERROR: Could not parse any markers")
            print("The file format might be different than expected")
            return None, None
        
        # Visualize the skeleton
        print(f"\n=== 3D VISUALIZATION ===")
        print(f"Visualizing {len(skeleton)} markers")
        
        fig = visualize_skeleton_3d(skeleton, f"Vicon Skeleton - Frame {frame}")
        
        # Print summary by body region
        regions = {}
        for marker, data in skeleton.items():
            region = data['anatomical_name'].split('_')[0]
            if region not in regions:
                regions[region] = []
            regions[region].append(marker)
        
        print(f"\n=== MARKER SUMMARY ===")
        for region, markers in regions.items():
            print(f"{region}: {len(markers)} markers")
        
        return skeleton, fig
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Simple test function
def quick_test(calib_data):
    """
    Quick test to see what markers are available
    """
    # calib_data = pd.read_csv(calib_file_path)
    
    anatomical_mapping = create_complete_anatomical_mapping()
    available_markers = []
    
    print("=== QUICK MARKER CHECK ===")
    for col_idx in range(len(calib_data.columns)):
        marker_name = calib_data.iloc[0, col_idx]
        if pd.notna(marker_name) and marker_name in anatomical_mapping:
            available_markers.append(marker_name)
    
    print(f"Found {len(available_markers)} markers in file:")
    for marker in sorted(available_markers):
        print(f"  - {marker} -> {anatomical_mapping[marker]}")
    
    return available_markers

In [36]:
# First, do a quick test to see what markers are available
# available_markers = quick_test(calib_data)

# Then process the full file
skeleton, fig = process_vicon_calibration(calib_data)

# If you want to see the raw data structure
# calib_data = pd.read_csv(calib_data)
analyze_vicon_structure(calib_data)

PROCESSING VICON CALIBRATION FILE
=== VICON FILE STRUCTURE ANALYSIS ===
Total rows: 403
Total columns: 571

--- Row 0 ---
Col 0: 's'
Col 1: 'mm'
Col 2: 'mm'
Col 3: 'mm'
Col 4: 'mm'
Col 5: 'mm'
Col 6: 'mm'
Col 7: 'mm'
Col 8: 'mm'
Col 9: 'mm'
Col 10: 'mm'
Col 11: 'mm'
Col 12: 'mm'
Col 13: 'mm'
Col 14: 'mm'
Col 15: 'mm'
Col 16: 'mm'
Col 17: 'mm'
Col 18: 'mm'
Col 19: 'mm'
Markers in row 0: ['s', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm', 'mm']

--- Row 1 ---
Col 1: 'X'
Col 2: 'Y'
Col 3: 'Z'
Col 4: 'X'
Col 5: 'Y'
Col 6: 'Z'
Col 7: 'X'
Col 8: 'Y'
Col 9: 'Z'
Col 10: 'X'
Col 11: 'Y'
Col 12: 'Z'
Col 13: 'X'
Col 14: 'Y'
Col 15: 'Z'
Col 16: 'X'
Col 17: 'Y'
Col 18: 'Z'
Col 19: 'X'

--- Row 2 ---
Col 0: '0'
Col 1: '144.462'
Col 2: '-42.247'
Col 3: '1523.584'
Col 4: '138.731'
Col 5: '101.001'
Col 6: '1531.458'
Col 7: '272.368'
Col 8: '-29.649'
Col 9: '1485.246'
Col 10: '266.168'
Col 11: '95.581'
Col 12: '1496.422'
Col 13: '229.676'
Col

In [34]:
calib_data

,Time,LFHD,Unnamed: 2,Unnamed: 3,RFHD,Unnamed: 5,Unnamed: 6,LBHD,Unnamed: 8,Unnamed: 9,...,Unnamed: 561,RThoraxAngles,Unnamed: 563,Unnamed: 564,LHeadAngles,Unnamed: 566,Unnamed: 567,RHeadAngles,Unnamed: 569,Unnamed: 570
0,s,mm,mm,mm,mm,mm,mm,mm,mm,mm,...,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg
1,NaN,X,Y,Z,X,Y,Z,X,Y,Z,...,Z,X,Y,Z,X,Y,Z,X,Y,Z
2,0,144.462,-42.247,1523.584,138.731,101.001,1531.458,272.368,-29.649,1485.246,...,-2.26,5.166,-3.373,2.26,0.359,-2.829,-2.14,0.359,2.829,2.14
3,0.01,144.623,-42.03,1523.559,138.768,101.04,1531.491,272.366,-29.598,1485.245,...,-2.262,5.159,-3.381,2.262,0.233,-2.767,-2.07,0.233,2.767,2.07
4,0.02,144.342,-42.21,1523.587,138.719,101.06,1531.426,272.343,-29.606,1485.25,...,-2.265,5.149,-3.381,2.265,0.335,-2.818,-2.163,0.335,2.818,2.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,3.96,146.121,-42.098,1524.065,139.844,101.503,1531.42,272.851,-30.128,1485.634,...,-2.046,4.785,-3.343,2.046,0.419,-2.457,-1.79,0.419,2.457,1.79
399,3.97,146.198,-42.076,1523.922,140.083,101.542,1531.387,272.707,-30.169,1485.595,...,-2.032,4.814,-3.335,2.032,0.401,-2.441,-1.816,0.401,2.441,1.816
400,3.98,146.312,-42.092,1523.871,139.926,101.514,1531.4,274.254,-29.821,1485.831,...,-2.022,4.835,-3.34,2.022,0.334,-2.399,-1.844,0.334,2.399,1.844
401,3.99,146.291,-41.677,1523.943,140.029,101.568,1531.338,272.846,-30.202,1485.659,...,-2.305,4.87,-3.396,2.305,0.303,-2.419,-1.863,0.303,2.419,1.863
